<a href="https://colab.research.google.com/github/Amplil/pytorch/blob/master/7_5_IMDb_Dataset_DataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.5 IMDb（Internet Movie Database）からDataLoaderを作成

- 本ファイルでは、IMDb（Internet Movie Database）のデータを使用して、感情分析（0：ネガティブ、1：ポジティブ）を2値クラス分類するためのDatasetとDataLoaderを作成します。


※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.5 学習目標

1.	テキスト形式のファイルデータからtsvファイルを作成し、torchtext用のDataLoaderを作成できるようになる

# 事前準備
書籍の指示に従い、本章で使用するデータを用意します


# 1. IMDbデータセットをtsv形式に変換

Datasetをダウンロードします

※torchtextで標準でIMDbが使える関数があるのですが、今回は今後データセットが用意されていない場合でも対応できるように0から作ります。

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

5万件のデータ（train,testともに2.5万件）です。データidとrating（1-10）でファイル名が決まっています。

rateは10の方が良いです。4以下がnegative、7以上がpositiveにクラス分けされています。



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/My Drive/Colab Notebooks/pytorch_advanced/7_nlp_sentiment_transformer"

/content/drive/My Drive/Colab Notebooks/pytorch_advanced/7_nlp_sentiment_transformer


In [3]:
ls "/content/drive/My Drive/Colab Notebooks/pytorch_advanced/data/aclImdb/train"

neg/  pos/  unsup/


In [4]:
# tsv形式のファイルにします
import glob
import os
import io
import string


# 訓練データのtsvファイルを作成します

f = open('./data/IMDb_train.tsv', 'w')

path = './../data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = './../data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


In [5]:
# テストデータの作成

f = open('./data/IMDb_test.tsv', 'w')

path = './../data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './../data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


In [7]:
text

'It\'s not just that the movie is lame. It\'s more than that. This movie is just unnecessary. Do we need another Western? How about a western with afro-Americans in the titles roles? Sound stupid, implausible and a lame attempt at modernizing the genre? It is. Incredibly lame and simple minded. It\'s like that lame Baz Luhrman film "Romeo and Juliet" where he set it in modern times to attract young folks and create some hype with his revamping of a classic tale. Well, Baz Luhrman failed miserably and so does this mess. The story is actually not bad however the whole idea of removing the racism out of a racist genre by casting an all afro-American cast is racist in itself. It\'s also puerile and simple minded (like Baz Luhrman-man he\'s a bad director). Hey (I hear you say) this was directed by Mario Van Peebles! He\'s also IN the film! How can it be racist? It\'s not. I said the idea of casting all afro-Americans instead of Caucasians was. The film isn\'t racist, it\'s just pointless, 

# 2. 前処理と単語分割の関数を定義

In [8]:
import string
import re

# 以下の記号はスペースに置き換えます（カンマ、ピリオドを除く）。
# punctuationとは日本語で句点という意味です
print("区切り文字：", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 前処理


def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）


def tokenizer_punctuation(text):
    return text.strip().split()


# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 動作を確認します
print(tokenizer_with_preprocessing('I like cats.'))


区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


# DataLoaderの作成

In [9]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
import torchtext


# 文章とラベルの両方に用意します
max_length = 256
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

# 引数の意味は次の通り
# init_token：全部の文章で、文頭に入れておく単語
# eos_token：全部の文章で、文末に入れておく単語


In [15]:
# フォルダ「data」から各tsvファイルを読み込みます
train_val_ds, test_ds = torchtext.legacy.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 動作確認
print('訓練および検証のデータ数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))


訓練および検証のデータ数 25116
1つ目の訓練および検証のデータ {'Text': ['wes', 'craven', 'has', 'been', 'created', 'a', 'most', 'successful', 'killer', 'thriller', 'movies', 'of', 'all', 'time', '.', 'after', 'watching', 'he', 's', 'movies', ',', 'you', 'will', 'find', 'your', 'new', 'fears', '.', 'people', 'don', 't', 'know', ',', 'which', 'wes', 'craven', 's', 'thriller', 'movie', 'is', 'the', 'best', ',', 'because', 'they', 'all', 'different', '.', 'in', 'this', 'movie', ',', 'lisa', 'is', 'terrorize', 'by', 'fellow', 'traveler', '.', 'he', 'coercible', 'her', 'to', 'kill', 'and', 'if', 'she', 'don', 't', 'do', 'this', ',', 'jack', 'will', 'kill', 'her', 'father', '.', 'lisa', 'is', 'in', 'the', 'huge', 'mess', ',', 'because', 'whatever', 'she', 'choose', ',', 'she', 'will', 'kill', '.', 'acting', 'was', 'unreal', '.', 'rachel', 'mcadams', 'and', 'cillian', 'murphy', 'acted', 'unbelievable', 'good', '.', 'the', 'emotions', 'was', 'in', 'right', 'choose', '.', 'idea', 'and', 'script', 'of', 'this', 'movie', 'i

In [16]:
train_val_ds

In [17]:
test_ds

In [22]:
ls

7-1_Tokenizer.ipynb                7-7_transformer_training_inference.ipynb
7-2_torchtext.ipynb                data/
7-4_vectorize.ipynb                make_folders_and_data_downloads7.ipynb
7-5_IMDb_Dataset_DataLoader.ipynb  utils/
7-6_Transformer.ipynb


In [12]:
torchtext.legacy.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

(<torchtext.legacy.data.dataset.TabularDataset at 0x7f146d4b1c50>,
 <torchtext.legacy.data.dataset.TabularDataset at 0x7f14bec98710>)

In [19]:
len(train_val_ds)

25116

In [14]:
ls ./data

IMDb_test.tsv  IMDb_train.tsv  text_test.tsv  text_train.tsv  text_val.tsv


In [20]:
import random
# torchtext.legacy.data.Datasetのsplit関数で訓練データとvalidationデータを分ける

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('1つ目の訓練データ', vars(train_ds[0]))


訓練データの数 20093
検証データの数 5023
1つ目の訓練データ {'Text': ['you', 'can', 't', 'really', 'go', 'far', 'when', 'the', 'initial', 'story', 'isn', 't', 'all', 'that', 'great', '.', 'the', 'premise', 'of', 'cyborg', 's', 'needing', 'blood', 'is', 'just', 'dopey', '.', 'the', 'script', 'is', 'blasé', '.', 'the', 'actors', 'don', 't', 'have', 'much', 'to', 'work', 'with', '.', 'the', 'sets', 'were', 'staged', 'out', 'in', 'the', 'desert', 'to', 'cut', 'costs', '.', 'it', 's', 'a', 'trademark', 'that', 'if', 'the', 'background', 'is', 'the', 'desert', ',', 'then', 'the', 'movie', 'has', 'no', 'budget', '.', 'lack', 'of', 'budget', 'is', 'okay', ',', 'if', 'there', 's', 'a', 'story', '.', 'solarbabies', 'and', 'blood', 'of', 'champions', 'are', 'examples', 'of', 'decent', 'work', 'from', 'no', '.', 'but', 'this', 'movie', 'looks', 'as', 'if', 'they', 'had', 'to', 'scrape', 'their', 'change', 'together', 'just', 'to', 'buy', 'the', 'cameraman', 'a', 'sandwich', '.', 'again', ',', 'forgivable', 'if', 'only',

# ボキャブラリーを作成

In [25]:
# torchtextで単語ベクトルとして英語学習済みモデルを読み込みます

from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='./../data/wiki-news-300d-1M.vec')


# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", english_fasttext_vectors.dim)
print("単語数：", len(english_fasttext_vectors.itos))


100%|█████████▉| 999447/999994 [01:55<00:00, 8392.48it/s]

1単語を表現する次元数： 300
単語数： 999994


In [26]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 17916個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


torch.Size([17892, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f140663cb10>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'the': 4,
             '.': 5,
             ',': 6,
             'and': 7,
             'a': 8,
             'of': 9,
             'to': 10,
             'is': 11,
             'it': 12,
             'in': 13,
             'i': 14,
             'this': 15,
             'that': 16,
             's': 17,
             'was': 18,
             'as': 19,
             'for': 20,
             'movie': 21,
             'with': 22,
             'but': 23,
             'film': 24,
             't': 25,
             'you': 26,
             'on': 27,
             'not': 28,
             'he': 29,
             'are': 30,
             'his': 31,
             'have': 32,
             'one': 33,
             'be': 34,
             'all': 35,
             'at': 36,
             'they': 37,

In [27]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.legacy.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.legacy.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.legacy.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[   2,    0,    6,  ...,  105,  151,    3],
        [   2,  180,  285,  ...,    1,    1,    1],
        [   2,   15,   24,  ...,  241,    6,    3],
        ...,
        [   2,   91,   10,  ...,   18,  179,    3],
        [   2,   14,  258,  ...,    1,    1,    1],
        [   2,   13, 7454,  ..., 1115, 1633,    3]]), tensor([256, 207, 256, 256, 201, 256, 137, 256, 256, 256, 156, 256, 115, 250,
        159, 193, 192, 251, 256, 185, 256, 256, 135, 256]))
tensor([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0])


このようにDataLoaderは単語のidを格納しているので、分散表現はディープラーニングモデル側でidに応じて取得してあげる必要があります。

ここまでの内容をフォルダ「utils」のdataloader.pyに別途保存しておき、次節からはこちらから読み込むようにします

以上